In [ ]:
# Mount to Google Drive
from google.colab import drive
import pandas as pd
import glob
import numpy as np


drive.mount('/content/drive', force_remount=True)

# Define Project Folder
FOLDERNAME = 'Colab\ Notebooks/Poster/raw_data'

%cd drive/MyDrive/$FOLDERNAME


Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Poster/raw_data


NVDA股價(NVDA)*已做log return
------------
黃金期貨(GOLD) daily*已做log return

美元指數(DTWEXBGS) daily*已做log return

那指100(NASDAQ100) daily*已做log return

非農就業指數(PAYEMS) monthly*已做YOY

消費者信心指數(UMCSENT) monthly*已做YOY

核心物價指數/通膨指數(CORESTICKM159SFRBATL) monthly

Real GDP(GDPC1) quarterly*已做YOY

西德州原油指數WTI(DCOILWTICO) daily*已做log return

比特幣(CBBTCUSD) daily*已做log return

VIX恐慌指數(VIXCLS) daily

初領失業補助人次(ICSA) weekly*已做YOY

連續失業補助人次(CCSA) weekly*已做YOY

In [ ]:
# Define device
import torch

if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print(device)

cuda


In [ ]:
def calculate_yoy(df, periods):
  """
  Calculate the YOY growth rate by given periods data.
  """
  df = df.sort_index()
  # use historical data to calculate YOY growth rate
  shifted_df = df.shift(periods)
  yoy = (df - shifted_df) / shifted_df
  # align YOY data to original data
  yoy = yoy.reindex(df.index)
  return yoy

In [ ]:
# read nvda csv
nvda = pd.read_csv('NVDA.csv')

# date tranform(csv.DATE is string), using as index
nvda['DATE'] = pd.to_datetime(nvda['DATE'], errors='coerce')
nvda.set_index('DATE', inplace=True)
nvda_index = nvda.index[1:] # to calculate the first data

# NVDA_PRICE's log return
nvda_price = pd.to_numeric(nvda['NVDA_Price'], errors='coerce')
nvda_logr = np.log(nvda_price).diff(1)
nvda_logr.name = 'NVDA_LR'
nvda_logr = nvda_logr.reindex(nvda_index)

# other csv
dataframes = []
full_date_range = pd.date_range(start='2013/01/20', end='2024/08/30', freq='D')

file_paths = glob.glob('*.csv')
file_paths.remove('NVDA.csv')

for file in file_paths:
  # read other csv
  df = pd.read_csv(file)

  # date tranform(csv.DATE is string)
  df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')
  df.set_index('DATE', inplace=True)

  # Gold['CLOSE'] need to be log return
  if file == 'GOLD.csv':
    # data type is different from nvda
    gold_close = df['G_CLOSE'].str.replace(',', '').astype(float)
    gold_logr = np.log(gold_close).diff(1)
    gold_logr.name = 'GOLD_LR'
    gold_logr = gold_logr.reindex(full_date_range).fillna(method='ffill')
    gold_logr = gold_logr.reindex(nvda_index)
    # append gold_lr to dataframes list
    dataframes.append(gold_logr)
  else:
    column_name = df.columns[0]
    if file in ['DCOILWTICO.csv', 'CBBTCUSD.csv', 'NASDAQ100.csv', 'DTWEXBGS.csv']:
      # DCOILWTICO, CBBTCUSD, NASDAQ100, DTWEXBGS need to be log return, too
      df[column_name] = df[column_name].astype(str).replace('.', np.nan).str.replace(',', '', regex=False).astype(float)
      df = np.log(df[column_name]).diff(1)
      df.name = column_name.replace(',', '') + '_LR'
    elif file in ['ICSA.csv', 'CCSA.csv']:  # weekly
      df = df.iloc[:, 0].astype(float)
      df = calculate_yoy(df, periods=52)
      df.name = column_name.replace(',', '') + '_YOY'
    elif file in ['PAYEMS.csv', 'UMCSENT.csv']:  # monthly
      df = df.iloc[:, 0].astype(float)
      df = calculate_yoy(df, periods=12)
      df.name = column_name.replace(',', '') + '_YOY'
    elif file == 'GDPC1.csv': # quarterly
      df = df.iloc[:, 0].astype(float)
      df = calculate_yoy(df, periods=4)
      df.name = column_name.replace(',', '') + '_YOY'
    # let VIXCLS divided by 100
    elif file == 'VIXCLS.csv':
      df[column_name] = df[column_name].replace('.', np.nan)  # '.' inplace to NaN
      df[column_name] = df[column_name].str.replace(',', '', regex=False).astype(float) # ',' inplace to ''
      df /= 100
    # the last csv: CORESTICKM159SFRBATL
    # all need to reindex and fillna(avoid missing data caused by data production day was the stockmarket closing day)
    df.replace('.', np.nan, inplace=True)
    df = df.reindex(full_date_range).fillna(method='ffill')
    df = df.reindex(nvda_index)
    # append df to dataframe list
    dataframes.append(df)

# merge nvda and dataframes
dataframes.insert(0, nvda_logr)
merge = pd.concat(dataframes, axis=1)

# saved as csv
merge.to_csv('1_data.csv')

<ipython-input-8-e14f7f06e06b>:67: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.reindex(full_date_range).fillna(method='ffill')
<ipython-input-8-e14f7f06e06b>:67: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.reindex(full_date_range).fillna(method='ffill')
<ipython-input-8-e14f7f06e06b>:67: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.reindex(full_date_range).fillna(method='ffill')
<ipython-input-8-e14f7f06e06b>:67: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.reindex(full_date_range).fillna(method='ffill')
<ipython-input-8-e14f7f06e06b>:36: FutureWarning: Series.fillna with 'method' is deprecated a